In [26]:
import mxnet as mx
from mxnet import gluon,autograd,contrib,image,nd,init
from mxnet.gluon import data as gdata,loss as gloss,nn
import gluoncv
import matplotlib as mpl
import matplotlib.pyplot as plt
import sys
import time
import os
import shutil
import sys
import numpy as np
import pandas as pd

# 首先进行数据预处理，整理数据集

In [27]:
sys.path.insert(0,'../')

In [28]:
import model

In [29]:
net = model.resnet50_cifar10()
net.initialize(init=init.Xavier())
net.collect_params().reset_ctx([mx.gpu()])

In [30]:
train_dataset = gluon.data.vision.ImageFolderDataset(root='../CIFAR10/data/kaggle_cifar10/train_valid_test/train')
test_dataset = gluon.data.vision.ImageFolderDataset(root='../CIFAR10/data/kaggle_cifar10/train_valid_test/valid')

In [31]:
transformer  = gluon.data.vision.transforms.Compose([gdata.vision.transforms.ToTensor()])

In [32]:
test_loader = gdata.DataLoader(test_dataset.transform_first(transformer),batch_size=16,shuffle=False,last_batch='keep')

In [33]:
def split_and_load_data(batch, ctx_list, batch_size):
    """

    :param batch: 
    :param ctx_list: 
    :param batch_size: 
    :return:
     new_batch:list of NDArray [[data1,data2,data3],[label1,label2,label3]]
        result of split data for each gpus
    """
    num_ctx = len(ctx_list)
    num_sample_pre_batch = batch_size // num_ctx
    # total_batch = batch_size*num_ctx
    new_batch = []
    # split one mini-batch to each ctx

    for i, data in enumerate(batch):
        new_data = []
        for j, ctx in enumerate(ctx_list):
            begin = j * num_sample_pre_batch
            end = min((j + 1) * num_sample_pre_batch, batch_size)
            split_data = mx.nd.slice_axis(data, axis=0, begin=begin, end=end)
            new_data.append(split_data.as_in_context(ctx))
        new_batch.append(new_data)
    return new_batch

In [34]:
def inference(net, train_dataset, test_dataset, test_dataloader, ctx_list):
    """Test pipeline"""
    test_preds = []

    net.hybridize(static_alloc=True)
    for i, batch in enumerate(test_dataloader):
        batch_size = len(batch[0])
        batch = split_and_load_data(batch, ctx_list, batch_size)
        preds_label = []
        for data, _ in zip(*batch):
            pred_score = net(data)
            pred_score = mx.nd.argmax(pred_score, axis=-1).astype(int).asnumpy()
            preds_label.extend(pred_score)
        print('complete {} batch'.format(i))
        test_preds.extend(preds_label)
    
    print(len(test_preds))
    
    sorted_ids = list(range(1, len(test_dataset) + 1))
    sorted_ids.sort(key=lambda x: str(x))
    df = pd.DataFrame({'id': sorted_ids, 'label': test_preds})
    df['label'] = df['label'].apply(lambda x: train_dataset.synsets[int(x)])
    
    csv_file_path = '_submission.csv'
    df.to_csv(csv_file_path, sep=',', index=False)

In [35]:
inference(net,train_dataset,test_dataset,test_loader,[mx.gpu()])

complete 0 batch
complete 1 batch
complete 2 batch
complete 3 batch
complete 4 batch
complete 5 batch
complete 6 batch
complete 7 batch
complete 8 batch
complete 9 batch
complete 10 batch
complete 11 batch
complete 12 batch
complete 13 batch
complete 14 batch
complete 15 batch
complete 16 batch
complete 17 batch
complete 18 batch
complete 19 batch
complete 20 batch
complete 21 batch
complete 22 batch
complete 23 batch
complete 24 batch
complete 25 batch
complete 26 batch
complete 27 batch
complete 28 batch
complete 29 batch
complete 30 batch
complete 31 batch
complete 32 batch
complete 33 batch
complete 34 batch
complete 35 batch
complete 36 batch
complete 37 batch
complete 38 batch
complete 39 batch
complete 40 batch
complete 41 batch
complete 42 batch
complete 43 batch
complete 44 batch
complete 45 batch
complete 46 batch
complete 47 batch
complete 48 batch
complete 49 batch
complete 50 batch
complete 51 batch
complete 52 batch
complete 53 batch
complete 54 batch
complete 55 batch
co